# Video Codec Unit (VCU) Demo Example: TRANSCODE -> FILE 

Video Codec Unit (VCU) in ZynqMP SOC is capable of encoding and decoding AVC/HEVC compressed video streams in real time.

This notebook example shows video transcoding usecase - the process of converting from one encoding format to another format. This example reads compressed file, decode it using VCU and again encode it to different codec and save the file in different format. For video decoding & re-encoding it uses VCU while in case of audio(optional), it uses software Gstreamer element. 

# Block Diagram

<img src="pictures/block-diagram-transcode-file.png" align="center" alt="Drawing" style="width: 600px; height: 200px"/>

# Test Setup 

1. Connect Ethernet cable. Check Internet connectivity. It is required for downloading compressed file from web-server.
2. Connect serial cable to monitor logs on serial console.
3. Connect USB camera with board.
4. If Board is connected to private network, then export proxy settings in /home/root/.bashrc file as below,      
    - create/open a bashrc file using "vi ~/.bashrc" 
        - Insert below line to bashrc file
            - export http_proxy="< private network proxy address >"
            - export https_proxy="< private network proxy address >"
        - Save and close bashrc file.

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Configuration settings 

In [2]:
from ipywidgets import interact
import ipywidgets as widgets
from common import common_vcu_demo_transcode_to_file
import os
from ipywidgets import HBox, VBox, Text, Layout

### Insert file path

In [3]:
input_path=widgets.Text(value='',
    placeholder='Insert file path',
    description='Input File:',
    #style={'description_width': 'initial'},
    disabled=False)
output_path=widgets.Text(value='',
    placeholder='(optional) /mnt/sata/op.h265',
    description='Output Path:',
    disabled=False)
HBox([input_path, output_path])

### Transcode

In [4]:
codec_type=widgets.RadioButtons(
    options=['avc', 'hevc'],
    description='Video Codec:',
    disabled=False)
audio_codec_type=widgets.RadioButtons(
    options=['none', 'aac', 'vorbis'],
    description='Audio Codec:',
    disabled=False)
HBox([codec_type, audio_codec_type])

# Advanced options:

In [5]:
bit_rate=widgets.Text(value='',
    placeholder='(optional) 1000, 20000',
    description='Bit Rate(Kbps):',
    style={'description_width': 'initial'},
    disabled=False)
periodicity_idr=widgets.Text(value='',
    placeholder='(optional) 30, 40, 50, etc',
    description='Periodicity Idr:',
    style={'description_width': 'initial'},
    #layout=Layout(width='33%', height='30px'),
    disabled=False)
gop_length=widgets.Text(value='',
    placeholder='(optional) 30, 60',
    description='Gop Length:',
    disabled=False)
HBox([periodicity_idr , gop_length, bit_rate])

In [6]:
entropy_buffers=widgets.Dropdown(
    options=['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'],
    value='5',
    description='Entropy Buffers Nos:',
    style={'description_width': 'initial'},
    disabled=False,)
show_fps=widgets.Checkbox(
    value=False,
    description='show-fps',
    #style={'description_width': 'initial'},
    disabled=False)
HBox([entropy_buffers, show_fps])
#HBox([periodicity_idr, output_path])

In [7]:
from IPython.display import clear_output
from IPython.display import Javascript

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

def clear_op(event):
    clear_output(wait=True)
    return

button1 = widgets.Button(
    description='Clear Output',
    style= {'button_color':'lightgreen'},
    #style= {'button_color':'lightgreen', 'description_width': 'initial'},
    layout={'width': '300px'}
)
button2 = widgets.Button(
    description='',
    style= {'button_color':'white'},
    #style= {'button_color':'lightgreen', 'description_width': 'initial'},
    layout={'width': '80px'},
    disabled=True
)
button1.on_click(run_all)
button1.on_click(clear_op)

In [8]:
def start_demo(event):
    #clear_output(wait=True)
    arg = common_vcu_demo_transcode_to_file.cmd_line_args_generator(input_path.value, bit_rate.value, codec_type.value, audio_codec_type.value, output_path.value, entropy_buffers.value, gop_length.value, periodicity_idr.value, show_fps.value);
    #!sh vcu-demo-transcode-to-file.sh $arg > logs.txt 2>&1
    !sh vcu-demo-transcode-to-file.sh $arg
    return

button = widgets.Button(
    description='click to start vcu-transcode-to-file demo',
    style= {'button_color':'lightgreen'},
    #style= {'button_color':'lightgreen', 'description_width': 'initial'},
    layout={'width': '300px'}
)
button.on_click(start_demo)
HBox([button, button2, button1])